SCOOTER RENTAL MANAGEMENT SYSTEM

In [12]:
import random
import uuid
from datetime import datetime

In [13]:
# Rental Period Enumeration
class RentalPeriod:
    HOURLY = 5    # $5 per scooter per hour
    DAILY = 20    # $20 per scooter per day
    WEEKLY = 50   # $50 per scooter per week


In [14]:
# Payment Processing Interface 
class IPaymentProcessor:
    def process_payment(self, amount: float) -> bool:
        #abstarct class with common structure whre dervived classes follow
        raise NotImplementedError("Derivedclass must implement abstract method")

#  Payment Processor
class CreditCardPaymentProcessor(IPaymentProcessor):
    def process_payment(self, amount: float) -> bool:
        print(f"Processing payment of ${amount:.2f}")
        return True

In [15]:
#Individual Scooter Management
class Scooter:
    def __init__(self,scooter_type: str):
        self.id = str(uuid.uuid4())
        self.type = scooter_type
        self.ride_count = 0
        self.is_available = True
        self.maintenance_log: list[datetime] = []  # records maintenance of scooters time stamps

    def increment_rides_count(self): 
        # ride count of the scooter is increased every time this function is called
        self.ride_count += 1

            
    def needs_maintenance(self) -> bool:
        # maintenance is checked every time this function is called
        return self.ride_count >= 10
    
    def reset_maintenance(self):
        # scooter is maintained by setting ride_count to zero and the maintainace time stamp is recorded
        self.ride_count = 0
        self.maintenance_log.append(datetime.now())

In [16]:
# Standard and Premium Scooter Subclasses
class StandardScooter(Scooter):
    def __init__(self):
        super().__init__("Standard")

class PremiumScooter(Scooter):
    def __init__(self):
        super().__init__("Premium")

In [ ]:
# Rental Tracking 
class RentalTransaction:
    def __init__(self, scooters: list[Scooter], rental_period, start_time):
        self.transaction_id = str(random.randint(1000, 9999)) # generates random transaction_id everytime a transaction is made 
        self.scooters = scooters
        self.rental_period = rental_period
        self.start_time = start_time
        self.end_time = None
        self.total_cost = 0.0
        self.is_active = True

    def calculate_cost(self):
        # Calculate base cost by multiplying period with no of scooters
        base_cost = self.rental_period * len(self.scooters) 
        # Apply 30% discount if the number of scooters are 3-5 scooters
        if 3 <= len(self.scooters) <= 5:
            base_cost *= 0.7
        self.total_cost = base_cost
        return base_cost

    def complete_transaction(self):
        #record the completion of transaction
        self.end_time = datetime.now()


In [ ]:
#Main System Class
class ScooterRental:
    def __init__(self, initial_standard_scooters: int = 10, initial_premium_scooters: int = 5):
        # initilazed the system with 10 standard scooters and 5 premium scooters
        self.scooters: list[Scooter] = []
        self.active_rentals: list[RentalTransaction] = []
        self.rental_history: list[RentalTransaction] = []
        self.payment_processor = CreditCardPaymentProcessor()

        # Dividing the inventory into standard and premium
        for _ in range(initial_standard_scooters):
            self.scooters.append(StandardScooter())
        
        for _ in range(initial_premium_scooters):
            self.scooters.append(PremiumScooter())

    def get_available_scooters(self, scooter_type: str = None)-> list[Scooter]:
        available = [scooter for scooter in self.scooters 
                     if scooter.is_available and not scooter.needs_maintenance()]
        #retriving only premium available scooters if scooter_type is mentioned as premium
        if scooter_type:
            available = [scooter for scooter in available if scooter.type == scooter_type]
        return available

    def rent_scooters(self, num_scooters:int, rental_period: RentalPeriod,scooter_type:str = None):
        if self.active_rentals:
            print("Cannot have multiple active rentals simultaneously")
            return None 
               
        available_scooters = self.get_available_scooters(scooter_type)
        if len(available_scooters) ==0:
            print("No Scooters available at the momemt!")
        elif len(available_scooters) < num_scooters:
            print("Insufficient number of scooters available.Choose fewer scooters")
            return None

        # Select and mark scooters as unavailable
        selected_scooters = available_scooters[:num_scooters]
        for scooter in selected_scooters:
            scooter.is_available = False

        # Create transaction and calculate total cost
        transaction = RentalTransaction(selected_scooters, rental_period, datetime.now())
        total_cost = transaction.calculate_cost()

        # Process payment
        if self.payment_processor.process_payment(total_cost):
            # Mark scooters as unavailable and increment ride count
            for scooter in selected_scooters:
                scooter.is_available = False
                scooter.increment_rides_count()
            
            self.active_rentals.append(transaction)
            return transaction

        return transaction

    def return_scooters(self, transaction: RentalTransaction):
        if not transaction.is_active:
            print("Transaction already completed.")
            return False        

        transaction.complete_transaction()

        # Mark scooters as available
        for scooter in transaction.scooters:
            scooter.is_available = True

        # Move from active rentals to history
        self.active_rentals.remove(transaction)
        self.rental_history.append(transaction)
        return True
    
    def perform_maintenance(self, scooter: Scooter):
        if scooter.needs_maintenance():
            print(f"Performing maintenance on Scooter {scooter.id} (Type: {scooter.type})")
            scooter.reset_maintenance()

    def generate_system_report(self):
        total_scooters = len(self.scooters)    
       

        print("\n--- SCOOTER RENTAL SYSTEM REPORTS ---")
        print(f"Total No. of Scooters: {total_scooters}")
        # Detailed scooter breakdown
        standard_scooters = [s for s in self.scooters if s.type == "Standard"]
        premium_scooters = [s for s in self.scooters if s.type == "Premium"]

        print(f"Standard Scooters: {len(standard_scooters)}")
        print(f"Premium Scooters: {len(premium_scooters)}")
        # Available scooters
        available_scooters = self.get_available_scooters()
        print(f"Available Scooters: {len(available_scooters)}")

        # Scooters requiring maintenance
        maintenance_scooters = [s for s in self.scooters if s.needs_maintenance()]
        print(f"Scooters Needing Maintenance: {len(maintenance_scooters)}")

        # Detailed maintenance information
        print("\nDetailed Scooter Maintenance Status:")
        for scooter in self.scooters:
            print(f"Scooter {scooter.id} (Type: {scooter.type}):")
            print(f"  Rides: {scooter.ride_count}")
            print(f"  Needs Maintenance: {scooter.needs_maintenance()}")
            if scooter.maintenance_log:
                print(f"  Last Maintenance: {scooter.maintenance_log[-1]}")
        
        active_rentals = len(self.active_rentals)
        print(f"Active Rentals: {active_rentals}")


        total_revenue = sum(transaction.total_cost for transaction in self.rental_history)
        print(f"Total Revenue Generated: ${total_revenue:.2f}")
        print("-----------------------------------\n")


In [ ]:
# Command-line interface for scooter rental system.
def main():
    rental_system = ScooterRental()

    while True:
        print("\n--- Scooter Rental  Management System ---")
        print("1. View Available No. of Scooters")
        print("2. Rent Scooters")
        print("3. Return Scooters")
        print("4. View System Report")
        print("5. Exit")

        choice = input("Enter your choice (1-5): ")

        if choice == '1':
            available_scooters = rental_system.get_available_scooters()
            print(f"\nAvailable Scooters: {len(available_scooters)}")
        

        elif choice == '2':
            num_scooters = int(input("Number of scooters to rent: "))
            period_choice = input("Rental Period (hourly:h /daily:d /weekly:w ): ").lower()
            
            try:
                if period_choice == 'h':
                    rental_period = RentalPeriod.HOURLY
                elif period_choice == 'd':
                    rental_period = RentalPeriod.DAILY
                elif period_choice == 'w':
                    rental_period = RentalPeriod.WEEKLY
                else:
                    raise ValueError("Invalid Period")
                
                scooter_type = "Premium" if input("Only premium scooters? (y/n): ").lower() == 'y' else "Standard"
                
                transaction = rental_system.rent_scooters(num_scooters, rental_period,scooter_type)
                if transaction:
                    print(f"Transaction created. Transaction ID: {transaction.transaction_id}")
            except ValueError:
                print("Invalid rental period. Try again")

        elif choice == '3':
            transaction_id = input("Enter the transactionID you would like to return: ")
            transaction = None
            for t in rental_system.active_rentals:
                if t.transaction_id == transaction_id:
                    transaction = t
                    break
            
            if transaction:
                if rental_system.return_scooters(transaction):
                    print(f"Rental returned. Total cost: ${transaction.total_cost:.2f}")
            else:
                print("Transaction not found!")

        elif choice == '4':
            rental_system.generate_system_report()
            # Perform maintenance on scooters that need it
            for scooter in rental_system.scooters:
                rental_system.perform_maintenance(scooter)

        elif choice == '5':
            print("Thank you for using Scooter Rental System!")
            break

        else:
            print("Invalid choice. Please try again.")


In [ ]:


if __name__ == "__main__":
    main()

''' 
Steps done to get the below output
1 Enter 1 to view available scooters
2 Enter 2 to and rent 3 standard scooters on hourly basis
3 Enter 4 to to view System report
4 Trying renting again (should not allow)
5 Enter 3 to return scooters based by transactionID. Total cost will be displayed
6 Enter 4 to view system report. Revenue gets updated along with ride count for respective scooters
7 Enter 5 to EXIT
'''



--- Scooter Rental  Management System ---
1. View Available No. of Scooters
2. Rent Scooters
3. Return Scooters
4. View System Report
5. Exit

Available Scooters: 15

--- Scooter Rental  Management System ---
1. View Available No. of Scooters
2. Rent Scooters
3. Return Scooters
4. View System Report
5. Exit
Processing payment of $10.50
Transaction created. Transaction ID: 9420

--- Scooter Rental  Management System ---
1. View Available No. of Scooters
2. Rent Scooters
3. Return Scooters
4. View System Report
5. Exit

--- SCOOTER RENTAL SYSTEM REPORTS ---
Total No. of Scooters: 15
Standard Scooters: 10
Premium Scooters: 5
Available Scooters: 12
Scooters Needing Maintenance: 0

Detailed Scooter Maintenance Status:
Scooter bbe8b95a-aea4-4f22-a5d0-307c363106c2 (Type: Standard):
  Rides: 1
  Needs Maintenance: False
Scooter d7f35494-22fd-4df2-867a-f8742d248cb1 (Type: Standard):
  Rides: 1
  Needs Maintenance: False
Scooter 371d6279-05c0-42cb-9fc1-bc67f74f6007 (Type: Standard):
  Rides: 1


' \nSteps done to get the below output\n1\n'

In [23]:
#Unit TestCases
class ScooterRentalTestSuite:
    def __init__(self):
        self.total_tests = 0
        self.passed_tests = 0
        self.failed_tests = 0

    # Method to compare values
    def assert_equal(self, actual, expected, message=""):
        self.total_tests += 1
        try:
            assert actual == expected, f"{message}\nExpected {expected}, but got {actual}"
            self.passed_tests += 1
            print(f"✓ Test Passed: {message}")
        except AssertionError as e:
            self.failed_tests += 1
            print(f"✗ Test Failed: {e}")

    # Method to test true if conditions
    def assert_true(self, condition, message=""):
        self.total_tests += 1
        try:
            assert condition, f"{message}"
            self.passed_tests += 1
            print(f"✓ Test Passed: {message}")
        except AssertionError as e:
            self.failed_tests += 1
            print(f"✗ Test Failed: {message}")

    # Method to chcek false if conditions
    def assert_false(self, condition, message=""):
        self.total_tests += 1
        try:
            assert not condition, f"{message}"
            self.passed_tests += 1
            print(f"✓ Test Passed: {message}")
        except AssertionError as e:
            self.failed_tests += 1
            print(f"✗ Test Failed: {message}")
    

    # Method to check Inventory
    def test_inventory(self):
        rental_system = ScooterRental()
        
        # Check total number of scooters
        self.assert_equal(len(rental_system.scooters), 15, "Initalially Total count should be 15")
        
        # Check number of standard scooters
        no_standard_scooters = len([s for s in rental_system.scooters if s.type == "Standard"])
        self.assert_equal(no_standard_scooters, 10, "There are 10 Standard scoters")
        
        # Check number of premium scooters
        no_premium_scooters = len([s for s in rental_system.scooters if s.type == "Premium"])
        self.assert_equal(no_premium_scooters, 5, "There are 5 premiun scooters")

    #Method to check available scooters function, scooter_type wise
    def test_available_scooters(self):
        rental_system = ScooterRental()
        
        # Get all available scooters
        available_all = rental_system.get_available_scooters()
        self.assert_equal(len(available_all), 15, "Avilable scooters not matching inventory")
        
        # Get only standard scooters
        available_standard = rental_system.get_available_scooters("Standard")
        self.assert_equal(len(available_standard), 10, "All standard scooters should be available")
        
        # Get only premium scooters
        available_premium = rental_system.get_available_scooters("Premium")
        self.assert_equal(len(available_premium), 5, "All premium scooters should be available")

    
    #Method to check total_cost
    def test_total_cost_calculations(self):
        rental_system = ScooterRental()
        
        # Test hourly rental without discount
        transaction_hourly_2 = rental_system.rent_scooters(2, RentalPeriod.HOURLY)
        self.assert_true(transaction_hourly_2 is not None, "Hourly rental for 2 scooters should be successful")
        self.assert_equal(transaction_hourly_2.total_cost, 2 * RentalPeriod.HOURLY, "Hourly rental cost calculation is incorrect")
        
        # Restart rental system to reset available scooters
        rental_system = ScooterRental()
        
        # Test daily rental with 30% discount (3-5 scooters)
        transaction_daily_4 = rental_system.rent_scooters(4, RentalPeriod.DAILY)
        self.assert_true(transaction_daily_4 is not None, "Daily rental for 4 scooters should be successful")
        
        expected_cost = 4 * RentalPeriod.DAILY * 0.7
        self.assert_equal(transaction_daily_4.total_cost, expected_cost, "Daily rental discount calculation incorrect")

    #Method to check scooter_maintenance check functionality
    def test_scooter_maintenance(self):
        rental_system = ScooterRental()
        scooter = rental_system.scooters[0]
        
        # Simulate 10 rides to trigger maintenance
        for _ in range(10):
            scooter.increment_rides_count()
        
        # Check maintenance requirement
        self.assert_true(scooter.needs_maintenance(), "Scooter should need maintenance after 10 rides")
        
        # Perform maintenance
        rental_system.perform_maintenance(scooter)
        
        # Verify maintenance reset
        self.assert_equal(scooter.ride_count, 0, "Ride count should be reset to 0 after maintenance")
        self.assert_false(scooter.needs_maintenance(), "Scooter should not need maintenance after reset")
        self.assert_true(len(scooter.maintenance_log) > 0, "Maintenance log should be updated")

    #Method to check transactional flow
    def test_rental_transaction_workflow(self):
        rental_system = ScooterRental()
        
        # Rent scooters
        transaction = rental_system.rent_scooters(2, RentalPeriod.WEEKLY)
        self.assert_true(transaction is not None, "Weekly rental should be successful")
        
        # Verify scooters are marked as unavailable
        for scooter in transaction.scooters:
            self.assert_false(scooter.is_available, "Rented scooters should be marked unavailable")
        
        # Return scooters
        return_result = rental_system.return_scooters(transaction)
        self.assert_true(return_result, "Scooter return should be successful")
        
        # Verify scooters are now available
        for scooter in transaction.scooters:
            self.assert_true(scooter.is_available, "Returned scooters should be marked available")

    #Run all test cases
    def run_tests(self):
        # Get all methods starting with 'test_' in the code 
        test_methods = [method for method in dir(self) if method.startswith('test_')]
        
        # Run each test method by looping through the array
        for method_name in test_methods:
            method = getattr(self, method_name)
            try:
                method()
            except Exception as e:
                print(f"✗ Test {method_name} encountered an error: {e}")
                self.failed_tests += 1
        
        # Print test summary
        print("\n--- TEST SUMMARY ---")
        print(f"Total Tests: {self.total_tests}")
        print(f"Passed: {self.passed_tests}")
        print(f"Failed: {self.failed_tests}")
        print("--------------------")

def main1():
    test_suite = ScooterRentalTestSuite()
    test_suite.run_tests()

if __name__ == "__main__":
    main1()

✓ Test Passed: Avilable scooters not matching inventory
✓ Test Passed: All standard scooters should be available
✓ Test Passed: All premium scooters should be available
✓ Test Passed: Initalially Total count should be 15
✓ Test Passed: There are 10 Standard scoters
✓ Test Passed: There are 5 premiun scooters
Processing payment of $100.00
✓ Test Passed: Weekly rental should be successful
✓ Test Passed: Rented scooters should be marked unavailable
✓ Test Passed: Rented scooters should be marked unavailable
✓ Test Passed: Scooter return should be successful
✓ Test Passed: Returned scooters should be marked available
✓ Test Passed: Returned scooters should be marked available
✓ Test Passed: Scooter should need maintenance after 10 rides
Performing maintenance on Scooter 6b860aec-04cb-4112-8fa9-27e32b49dd55 (Type: Standard)
✓ Test Passed: Ride count should be reset to 0 after maintenance
✓ Test Passed: Scooter should not need maintenance after reset
✓ Test Passed: Maintenance log should be 